<img src="https://raw.githubusercontent.com/rbizoi/PythonFormation/main/images/e-brasil.png" width="850">

# Les imports et initialisations des variables

In [1]:
from datetime import datetime
import pandas as pd, numpy as np, os, warnings, seaborn as sns, pickle, re, unicodedata
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font1 = fm.FontProperties(size=20)
font2 = fm.FontProperties(size=24)

%matplotlib inline

if int(str(sns.__version__).split('.')[1]) > 8 : 
    plt.style.use('seaborn-v0_8-darkgrid')
else:
    plt.style.use('seaborn-darkgrid')
    
sns.set(font_scale=2)
warnings.filterwarnings(action="ignore")

## Liste fichiers de données 

In [2]:
!dir ..\donnees\ebrasil

 Le volume dans le lecteur F s'appelle Data
 Le num‚ro de s‚rie du volume est D4F7-8E12

 R‚pertoire de F:\PythonFormation\donnees\ebrasil

14/02/2024  09:47    <DIR>          .
14/02/2024  09:47    <DIR>          ..
06/10/2019  20:27         9ÿ033ÿ957 olist_customers_dataset.csv
06/10/2019  20:27        61ÿ273ÿ883 olist_geolocation_dataset.csv
06/10/2019  20:27        17ÿ654ÿ914 olist_orders_dataset.csv
06/10/2019  20:27        15ÿ438ÿ671 olist_order_items_dataset.csv
06/10/2019  20:27         5ÿ777ÿ138 olist_order_payments_dataset.csv
06/10/2019  20:27        14ÿ409ÿ007 olist_order_reviews_dataset.csv
06/10/2019  20:27         2ÿ379ÿ446 olist_products_dataset.csv
28/09/2020  07:52           177ÿ799 olist_sellers_dataset.csv
17/01/2024  08:49             2ÿ542 product_category_name_translation.csv
               9 fichier(s)      126ÿ147ÿ357 octets
               2 R‚p(s)  86ÿ205ÿ255ÿ680 octets libres


## Changement de répertoire

In [3]:
os.chdir(r"../donnees")

# DataFrame $reviews$

In [4]:
donnees = pd.read_csv(os.path.join('ebrasil', 'olist_order_reviews_dataset.csv'))
donnees.rename(columns={col:col.replace('review_','') for col in donnees.columns[2:]},inplace=True)
donnees.creation_date = pd.to_datetime(donnees.creation_date, format='%Y-%m-%d %H:%M:%S',errors='ignore')
donnees.answer_timestamp = pd.to_datetime(donnees.answer_timestamp, format='%Y-%m-%d %H:%M:%S',errors='ignore')
donnees = donnees.merge(pd.read_parquet('./ecommerce/orders.parquet')[['order_id', 'purchase_timestamp']],on='order_id')
donnees['creation'] = (donnees.creation_date - donnees.purchase_timestamp).dt.seconds/60/60
donnees['answer'] = (donnees.answer_timestamp - donnees.creation_date).dt.seconds/60/60
donnees.score = donnees.score.astype('int16')
donnees['comment'] = donnees.comment_message.apply(lambda x : len(str(x))).astype('int16')
donnees.head()

,review_id,order_id,score,comment_title,comment_message,creation_date,answer_timestamp,purchase_timestamp,creation,answer,comment
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59,2018-01-11 15:30:49,8.486389,21.783056,3
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13,2018-02-28 12:25:19,11.578056,3.086944,3
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24,2018-02-03 09:56:22,14.060556,14.606667,3
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,2017-04-09 17:41:13,6.313056,22.035000,37
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,2018-02-10 10:59:03,13.015833,10.448056,100


In [5]:
donnees.order_id.nunique(),donnees.shape[0]

(99441, 100000)

In [6]:
donnees.review_id.nunique(),donnees.shape

(99173, (100000, 11))

In [7]:
donnees.dtypes

review_id                     object
order_id                      object
score                          int16
comment_title                 object
comment_message               object
creation_date         datetime64[ns]
answer_timestamp      datetime64[ns]
purchase_timestamp    datetime64[ns]
creation                     float64
answer                       float64
comment                        int16
dtype: object

In [8]:
donnees.to_parquet('./ecommerce/reviews.parquet',compression='gzip', engine='pyarrow')

In [9]:
donnees = donnees.pivot_table(index='order_id',
                    columns='score',
                    values=['review_id','creation','answer','comment'], 
                    aggfunc={'review_id':'count','creation':'sum','answer':'sum','comment':'sum'},
                    fill_value=0)
donnees.columns = [f"{col[0].replace('review_id','score')}_{col[1]}" for col in donnees.columns]
donnees.head()

,answer_1,answer_2,answer_3,answer_4,answer_5,comment_1,comment_2,comment_3,comment_4,comment_5,creation_1,creation_2,creation_3,creation_4,creation_5,score_1,score_2,score_3,score_4,score_5
order_id,,,,,,,,,,,,,,,,,,,,
00010242fe8c5a6d1ba2dd792cb16214,0.0,0.0,0.0,0.000000,10.950833,0,0,0,0,46,0.0,0.0,0.0,0.000000,15.016111,0,0,0,0,1
00018f77f2f0320c557190d7a144bdd3,0.0,0.0,0.0,11.570278,0.000000,0,0,0,3,0,0.0,0.0,0.0,13.115000,0.000000,0,0,0,1,0
000229ec398224ef6ca0657da4fc703e,0.0,0.0,0.0,0.000000,16.108611,0,0,0,0,90,0.0,0.0,0.0,0.000000,9.441389,0,0,0,0,1
00024acbcdf0a6daa1e931b038114c75,0.0,0.0,0.0,16.650278,0.000000,0,0,0,3,0,0.0,0.0,0.0,13.990278,0.000000,0,0,0,1,0
00042b26cf59d7ce69dfabb4e55b4fd9,0.0,0.0,0.0,0.000000,10.916389,0,0,0,0,39,0.0,0.0,0.0,0.000000,10.035833,0,0,0,0,1


In [10]:
donnees.to_parquet('./ecommerce/reviews_p.parquet',compression='gzip', engine='pyarrow')

In [11]:
donnees = pd.read_parquet('./ecommerce/orders_payments.parquet').merge(donnees,how='left',on='order_id')

In [12]:
donnees.head()

,order_id,customer_id,purchase_timestamp,approved_at,delivered_carrier,delivered_customer,estimated_delivery,status,annee,mois,...,creation_1,creation_2,creation_3,creation_4,creation_5,score_1,score_2,score_3,score_4,score_5
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,delivered,2017,10,...,0.0,0.0,0.0,13.057500,0.000000,0,0,0,1,0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,delivered,2018,7,...,0.0,0.0,0.0,3.306389,0.000000,0,0,0,1,0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,delivered,2018,8,...,0.0,0.0,0.0,0.000000,15.353056,0,0,0,0,1
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,delivered,2017,11,...,0.0,0.0,0.0,0.000000,4.531667,0,0,0,0,1
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,delivered,2018,2,...,0.0,0.0,0.0,0.000000,2.689167,0,0,0,0,1


In [13]:
donnees.to_parquet('./ecommerce/orders_payments_reviews_p.parquet',compression='gzip', engine='pyarrow')

In [14]:
!dir ecommerce

 Le volume dans le lecteur F s'appelle Data
 Le num‚ro de s‚rie du volume est D4F7-8E12

 R‚pertoire de F:\PythonFormation\donnees\ecommerce

14/02/2024  15:37    <DIR>          .
14/02/2024  15:37    <DIR>          ..
14/02/2024  15:45         4ÿ525ÿ883 customers.parquet
14/02/2024  15:37         1ÿ122ÿ602 geolocation.parquet
14/02/2024  15:46         4ÿ425ÿ246 items.parquet
14/02/2024  15:46         4ÿ527ÿ569 items_products.parquet
14/02/2024  15:45        27ÿ443ÿ040 orders.csv
14/02/2024  15:45         9ÿ269ÿ678 orders.parquet
14/02/2024  15:46         9ÿ725ÿ278 orders_payments.parquet
14/02/2024  15:47        11ÿ010ÿ093 orders_payments_reviews_p.parquet
14/02/2024  15:46         2ÿ375ÿ117 payments.parquet
14/02/2024  15:46           963ÿ000 products.parquet
14/02/2024  15:47         7ÿ858ÿ269 reviews.parquet
14/02/2024  15:47         3ÿ201ÿ554 reviews_p.parquet
14/02/2024  15:06            91ÿ808 sellers.parquet
              13 fichier(s)       86ÿ539ÿ137 octets
               2 R